In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
# manuplation hourly data to remove minutes
raw_data = pd.read_csv('raw_data.csv')
time_dt = pd.to_datetime(raw_data['time'], format='%d.%m.%Y %H:%M')
time_dt_hourbase = time_dt

for i in range(0, len(time_dt)):
    mmin = pd.Timedelta(minutes=time_dt[i].minute)
    time_dt_hourbase[i] = time_dt[i] - mmin
# Creating time domain for bus working hours
data_new = raw_data.set_index(time_dt_hourbase)
start_date = min(data_new.index)
end_date = max(data_new.index)

time_domain = pd.date_range(start=start_date, end=end_date, freq='H')
working_time_domain = time_domain[time_domain.hour > 6]
working_time_domain = working_time_domain[working_time_domain.hour < 17]
working_time_domain = pd.DataFrame(data=working_time_domain, columns=['rev_time'])
working_time_domain = working_time_domain.set_index(working_time_domain['rev_time'])
# seperating data set for each municiplaity ID and make domain same

data_new_sorted = data_new.sort_values(by=['user'], ascending=False)
data_cleared = pd.DataFrame()
for i in range(0, 10):
    temp_df = data_new_sorted[data_new_sorted['place_id'] == i]
    temp_ind = temp_df.index.duplicated(keep='first')
    last_df = temp_df[temp_ind == False]
    last_df = last_df.sort_index()
    last_df = pd.concat([working_time_domain, last_df], axis=1)
    last_df['place_id'] = last_df['place_id'].fillna(i)
    data_cleared = data_cleared.append(last_df, ignore_index=False)
# continiuty check
date_freq = pd.DataFrame()
for i in range(0, 10):
    y = data_cleared['user'][data_cleared['place_id'] == i]
    date_freq['place_' + str(i)] = pd.Series(working_time_domain.index.date[y.isna() == True]).value_counts()

# day should be assumed completely = 20/06, 21/06, 31/07,03/08,04/08
missing_dates = date_freq.index[date_freq['place_0'] == 10]
# other missing data are only one hour in a day
filled_data = pd.DataFrame()
for i in range(0, 10):
    y = data_cleared[data_cleared['place_id'] == i]
    for k in range(0, len(y)):
        if y.index[k].date() in missing_dates:
            y.time[k] = y.rev_time[k]
            y.user[k] = (y.user[(k - 70)]+ y.user[(k -140)])*0.5
            y.capacity[k] = y.capacity[k - 1]
        elif pd.isna(y.user[k]) and (y.index[k].hour == 7 or y.index[k].hour == 16):
            y.time[k] = y.rev_time[k]
            y.user[k] = (y.user[k - 10]+y.user[k-20])*0.5
            y.capacity[k] = y.capacity[k - 1]
        elif pd.isna(y.user[k]):
            y.time[k] = y.rev_time[k]
            y.user[k] = (y.user[k - 1] + y.user[k + 1]) * 0.5
            y.capacity[k] = y.capacity[k - 1]
    filled_data = filled_data.append(y, ignore_index=False)


In [4]:
print(filled_data)

                               rev_time              time  place_id   user  \
2017-06-04 07:00:00 2017-06-04 07:00:00  04.06.2017 07:59       0.0  204.0   
2017-06-04 08:00:00 2017-06-04 08:00:00  04.06.2017 08:59       0.0  332.0   
2017-06-04 09:00:00 2017-06-04 09:00:00  04.06.2017 09:59       0.0  485.0   
2017-06-04 10:00:00 2017-06-04 10:00:00  04.06.2017 10:59       0.0  583.0   
2017-06-04 11:00:00 2017-06-04 11:00:00  04.06.2017 11:59       0.0  614.0   
...                                 ...               ...       ...    ...   
2017-08-19 12:00:00 2017-08-19 12:00:00  19.08.2017 12:30       9.0  849.0   
2017-08-19 13:00:00 2017-08-19 13:00:00  19.08.2017 13:30       9.0  941.0   
2017-08-19 14:00:00 2017-08-19 14:00:00  19.08.2017 14:30       9.0  983.0   
2017-08-19 15:00:00 2017-08-19 15:00:00  19.08.2017 15:03       9.0  976.0   
2017-08-19 16:00:00 2017-08-19 16:00:00  19.08.2017 16:03       9.0  879.0   

                     capacity  
2017-06-04 07:00:00    2813.0  

In [15]:
#Model Comparison (Mean Absolute Percante(%) Error) for developped model fit to existing data and forecast for last two weeks
results= pd.read_csv('results.csv')
print(results)

   Place_id  Week77_Fourier_Forecast  Week77_MLR_Forecast  \
0         0                32.764482            30.666552   
1         1                17.017501             8.137491   
2         2                11.869993            10.890132   
3         3                19.448999            19.125181   
4         4                19.488098            21.222802   
5         5                43.209712            38.905481   
6         6                27.029943            22.850619   
7         7                 8.103196             7.673241   
8         8                12.445143            13.455173   
9         9                11.334480             8.659284   

   Week76_Fourier_Forecast  Week76_MLR_Forecast  Fourier_Model_Fit  \
0                12.600032            10.878098          18.539735   
1                 9.037163             7.086301          10.668371   
2                22.034328            14.768196          11.671205   
3                22.259353            18.487535 

In [ ]:
#Explanation & comments : I developped two model both based on regression but one of them coefficients comes from Fourier Expansion because if daily curves of the data are checked there is weekly periyodik cycle
#so Fourier expansion with modulation is powerful method to modelling this kind of data. Other model, multiple linear regression. When we compare results for forecast and modelling, Fourier Model's errors
#are the lower for modelling existing data while forecast errors are quite high.
#I didn't make any residual analysis because this just to show how I approach to any problem. 